# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [52]:
import requests
import pyowm
import json
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [53]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 286.79,
  'feels_like': 285.85,
  'temp_min': 286.79,
  'temp_max': 286.79,
  'pressure': 1022,
  'humidity': 63,
  'sea_level': 1022,
  'grnd_level': 1018},
 'visibility': 10000,
 'wind': {'speed': 3.03, 'deg': 26, 'gust': 3.73},
 'clouds': {'all': 75},
 'dt': 1745492405,
 'sys': {'country': 'GB', 'sunrise': 1745469938, 'sunset': 1745521877},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [54]:
from pyowm.owm import OWM
owm = OWM(api_key)
mgr = owm.weather_manager()
observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
weather.status           # short version of status (eg. 'Rain')
weather.detailed_status  # detailed version of status (eg. 'light rain')

'broken clouds'

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [55]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 286.79°C, Humidity: 63%, Wind Speed: 3.03 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [72]:
owm = OWM(api_key)
mgr = owm.weather_manager()
observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
temperature = weather.temperature()["temp"]           # short version of status (eg. 'Rain')
humidity = weather.humidity  # detailed version of status (eg. 'light rain')
wind_speed = weather.wind()["speed"]

print(f"Temperature: {temperature}°F, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 285.9°F, Humidity: 69%, Wind Speed: 3.03 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [58]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [59]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-04-24 12:00:00, Temp: 288.07°C
Time: 2025-04-24 15:00:00, Temp: 290.25°C
Time: 2025-04-24 18:00:00, Temp: 294.13°C
Time: 2025-04-24 21:00:00, Temp: 296.23°C
Time: 2025-04-25 00:00:00, Temp: 293.62°C
Time: 2025-04-25 03:00:00, Temp: 291.3°C
Time: 2025-04-25 06:00:00, Temp: 289.67°C
Time: 2025-04-25 09:00:00, Temp: 288.24°C
Time: 2025-04-25 12:00:00, Temp: 289.03°C
Time: 2025-04-25 15:00:00, Temp: 293.09°C
Time: 2025-04-25 18:00:00, Temp: 293.22°C
Time: 2025-04-25 21:00:00, Temp: 291.01°C
Time: 2025-04-26 00:00:00, Temp: 291.51°C
Time: 2025-04-26 03:00:00, Temp: 289.67°C
Time: 2025-04-26 06:00:00, Temp: 287.55°C
Time: 2025-04-26 09:00:00, Temp: 286.91°C
Time: 2025-04-26 12:00:00, Temp: 288.35°C
Time: 2025-04-26 15:00:00, Temp: 288.63°C
Time: 2025-04-26 18:00:00, Temp: 290.77°C
Time: 2025-04-26 21:00:00, Temp: 287.87°C
Time: 2025-04-27 00:00:00, Temp: 284.74°C
Time: 2025-04-27 03:00:00, Temp: 282.93°C
Time: 2025-04-27 06:00:00, Temp: 282.22°C
Time: 2025-04-27 09:00:00, Temp: 28

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [ ]:
#your code here
from pyowm.utils.config import get_default_config

# Set a higher timeout value
config_dict = get_default_config()
config_dict['connection']['timeout_secs'] = 15  # Increase timeout to 15 seconds

owm = OWM(api_key, config_dict)
mgr = owm.weather_manager()

# Fetch the daily forecast
try:
	forecast = mgr.forecast_at_place('Milan,IT', 'daily')
except Exception as e:
	print(f"An error occurred: {e}")

An error occurred: Invalid API Key provided


--> Does not work because owm changed access rights

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [94]:
data

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 286.12,
  'feels_like': 285.25,
  'temp_min': 286.12,
  'temp_max': 286.12,
  'pressure': 1023,
  'humidity': 68,
  'sea_level': 1023,
  'grnd_level': 1018},
 'visibility': 10000,
 'wind': {'speed': 3.03, 'deg': 26, 'gust': 3.73},
 'clouds': {'all': 75},
 'dt': 1745493813,
 'sys': {'country': 'GB', 'sunrise': 1745469938, 'sunset': 1745521877},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [97]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_fahrenheit = data['main']['temp']
temperature_celsius = (temperature_fahrenheit - 32) * 5/9

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 141.17777777777778°C, Fahrenheit: 286.12°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [96]:
#your code here
observation = mgr.weather_at_place('London')  # the observation object is a box containing a weather object
weather = observation.weather
temperature = weather.temperature('celsius')["temp"]          
humidity = weather.humidity  
wind_speed = weather.wind()["speed"]

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 12.97°C, Humidity: 68%, Wind Speed: 3.03 m/s
